In [41]:
# Importation des bibliothèques nécessaires
from pyspark.sql import SparkSession

In [42]:
# Variables
# Paramètres de connexion MySQL
db_url = ("jdbc:mysql://mysql-container:3306/wildlens"
          "?serverTimezone=UTC")
db_properties = {
    "user": "root",
    "password": "root",
    "driver": "com.mysql.cj.jdbc.Driver"
}

In [43]:
# Initialisation de SparkSession
spark = SparkSession.builder \
    .appName("WildLens ETL - MSPR 24-25") \
    .config("spark.jars", "/usr/lib/jvm/jars/mysql-connector-j-9.1.0.jar") \
    .getOrCreate()

In [44]:
# Chargement des informations sur les espèces
species_info_path = './data/csv/infos_especes.csv'
species_info_df = spark.read.csv(species_info_path, sep=";",header=True, inferSchema=True)
species_info_df.show(5)


+--------+--------------------+--------------------+---------+--------------------+--------------------+--------------------+--------------------+
|  Espèce|         Description|           Nom latin|  Famille|              Taille|              Région|             Habitat|            Fun fact|
+--------+--------------------+--------------------+---------+--------------------+--------------------+--------------------+--------------------+
|  Castor|Le castor d’Europ...|   Castor canadensis|Mammifère|100 à 135 cm queu...|Europe du nord et...|Le castor d’Europ...|À l’exception des...|
|    Chat|Le Chat sauvage4 ...|    Felis silvestris|Mammifère|46 à 51 cm sans l...|Amérique, Europe,...|                NULL|Un chat ne peut p...|
|   Chien|Alors qu'on estim...|Canis lupus famil...|Mammifère|                NULL|     Hémisphère nord|                NULL|L'empreinte de la...|
|  Coyote|Le coyote, égalem...|       Canis latrans|Mammifère|         70 à 120 cm|Amérique du nord,...|Le coyote vit 

In [45]:
# Remplacer les valeurs manquantes par "valeur manquante"
df_filled = species_info_df.fillna("missing value")

# Identifier les colonnes sans nom
columns_with_name = [col for col in df_filled.columns if col.strip()]

# Identifier les colonnes aberrantes (avec une seule valeur unique)
abnormal_columns = []
for col_name in columns_with_name:
    unique_value_count = df_filled.select(col_name).distinct().count()
    if unique_value_count <= 1:
        abnormal_columns.append(col_name)

# Supprimer les colonnes aberrantes
columns_to_keep = [col for col in columns_with_name if col not in abnormal_columns]
cleaned_df = df_filled.select(*columns_to_keep)

# Afficher un aperçu
cleaned_df.show()


+------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|      Espèce|         Description|           Nom latin|              Taille|              Région|             Habitat|            Fun fact|
+------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|      Castor|Le castor d’Europ...|   Castor canadensis|100 à 135 cm queu...|Europe du nord et...|Le castor d’Europ...|À l’exception des...|
|        Chat|Le Chat sauvage4 ...|    Felis silvestris|46 à 51 cm sans l...|Amérique, Europe,...|       missing value|Un chat ne peut p...|
|       Chien|Alors qu'on estim...|Canis lupus famil...|       missing value|     Hémisphère nord|       missing value|L'empreinte de la...|
|      Coyote|Le coyote, égalem...|       Canis latrans|         70 à 120 cm|Amérique du nord,...|Le coyote vit dan...|Animal social et ...|
|    Ecureuil

In [46]:
# Sauvegarde au format CSV
df_nettoye.write.csv("./output/data/nettoye.csv", header=True, mode="overwrite")

# Sauvegarde au format Parquet
df_nettoye.write.parquet("./output/data/nettoye.parquet", mode="overwrite")


In [47]:
# Écrire les données dans la table MySQL
df_nettoye.write \
    .jdbc(url=db_url, table="Especes", mode="overwrite", properties=db_properties)
